# ReAct agent with Wikipedia and calculator tool access

Trying to learn to make basic agents that can use tools. I found a YouTube video from June 2023 that used ReAct with Langchain to look stuff up with Wikipedia [(link)](https://www.youtube.com/watch?v=Eug2clsLtFs). However, the code used there doesn't work any more (all been deprecated) so I had to do most of this from scratch. I added in a calculator tool as well (which was a bit messy as I had to define a wrapper for the class so it would have a description attribute). Seems to work reasonably well now

## Importing libraries and setting up llm

In [3]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMMathChain
from langchain.tools import BaseTool

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o-mini")


/Users/nikmitchell/miniconda3/envs/langchain_react/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new AgentExecutor chain...
I need to find out which film Russell Crowe won his first Oscar for, who directed that film, and when the director was born. After gathering that information, I will then take the year of birth of the director and calculate its square root. 

Action: wikipedia  
Action Input: "Russell Crowe first Oscar film"  
Page: Russell Crowe
Summary: Russell Ira Crowe (born 7 April 1964) is a New Zealand-born actor. He was born in Wellington, spending 10 years of his childhood in Australia and residing there permanently by age 21. His work on screen has earned him various accolades, including an Academy Award, two Golden Globe Awards, and a British Academy Film Award.
Crowe began acting in Australia and had his breakout role in the drama Romper Stomper (1992). He gained international recognition for his starring roles as a police detective in the thriller L.A. Confidential (1997) and Jeffrey Wigand in the drama The Insider (1999). Crowe gained wider stardom 

## Defining wikipedia tool

In [ ]:

# Set up the Wikipedia tool
api_wrapper = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)


## Defining calculator tool

In [ ]:

# Create a proper tool wrapper by extending BaseTool
class MathToolWrapper(BaseTool):
    name: str = "Math Calculator"
    description: str = "A tool to perform mathematical calculations."
    _math_chain: LLMMathChain  # Private attribute

    def __init__(self, llm):
        super().__init__()
        self._math_chain = LLMMathChain(llm=llm)

    def _run(self, query: str):
        return self._math_chain.run(query)

    async def _arun(self, query: str):
        raise NotImplementedError("Async mode is not supported for this tool.")

# Initialize the math tool with the wrapper
math_tool = MathToolWrapper(llm=llm)


## Defining the agent

In [ ]:

tools = [wikipedia_tool, math_tool]

# Define the prompt template
prompt_template = PromptTemplate.from_template(
    '''Answer the following questions as best you can.
    You have access to the following tools:
    {tools}
    Use the following format:
    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question
    Begin!
    Question: {input}
    Thought:{agent_scratchpad}'''
)

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template
)

# Set up the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


## Asking it questions

In [ ]:

# Execute the agent with a specific query
# query = "How old is the president of the united states? Divide your answer by 3 and add 7.5"
query ="What is the first film that Russell Crowe won an Oscar for, and who directed that movie? When was that director born? Take your answer and square root it"
response = agent_executor.invoke({"input": query})
print(response)